In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from kobert_tokenizer import KoBERTTokenizer
import torch
import pytorch_lightning as pl
from transformers import BertModel
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import joblib
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
from torch.utils.data import DataLoader, TensorDataset


In [10]:
# 데이터 로드 및 전처리
data = pd.read_excel('./data/data.xlsx').iloc[:, :2]
X_train, X_test, y_train, y_test = train_test_split(data['Sentence'], data['Emotion'], test_size=0.2, random_state=42)

# 데이터 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [11]:
class BertFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('skt/kobert-base-v1')
        self.bert.eval()  # 평가 모드로 설정

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # 그래디언트 계산 비활성화
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state.mean(dim=1)

In [12]:
# 모델 초기화
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = BertFeatureExtractor().to(device)
print(device)

mps


In [15]:
sample_size = 100  # 원하는 샘플 크기
X_train_sample = X_train[:sample_size]
X_test_sample = X_test[:sample_size]

In [16]:
from tqdm import tqdm
# 특징 추출 함수
def get_embeddings(texts, model, device):
    model.eval()
    tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
    encodings = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=512, return_tensors='pt')
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=32)
    
    all_embeddings = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="특징 추출 중", unit="batch"):
            input_ids, attention_mask = [t.to(device) for t in batch]
            embeddings = model(input_ids, attention_mask)
            all_embeddings.append(embeddings.cpu())
    
    return torch.cat(all_embeddings, dim=0).numpy()

# 특징 추출
X_train_embeddings = get_embeddings(X_train_sample, model, device)
X_test_embeddings = get_embeddings(X_test_sample, model, device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
특징 추출 중: 100%|██████████| 4/4 [00:00<00:00,  5.66batch/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
특징 추출 중: 100%|██████████| 4/4 [00:01<00:00,  3.50batch/s]


In [20]:
y_train_sample = y_train[:sample_size]
y_test_sample = y_test[:sample_size]

In [21]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_sample)
y_test_encoded = label_encoder.transform(y_test_sample)

In [22]:
# 데이터 정규화 및 다중 나이브 베이즈 모델 훈련
print("나이브 베이즈 모델 훈련 중...")
pipeline = make_pipeline(MinMaxScaler(), MultinomialNB())
pipeline.fit(X_train_embeddings, y_train_encoded)

나이브 베이즈 모델 훈련 중...


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('multinomialnb', MultinomialNB())])

In [82]:
# 모델 평가
print("모델 평가 중...")
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test_embeddings)
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

# 모델 저장
print("모델 저장 중...")
joblib.dump(pipeline, 'naive_bayes_pipeline.joblib')
print("완료!")

모델 평가 중...
              precision    recall  f1-score   support

          공포       0.41      0.45      0.43      1096
          놀람       0.36      0.41      0.39      1177
          분노       0.40      0.44      0.42      1150
          슬픔       0.43      0.40      0.41      1020
          중립       0.29      0.13      0.18       974
          행복       0.57      0.68      0.62      1172
          혐오       0.30      0.29      0.29      1130

    accuracy                           0.41      7719
   macro avg       0.39      0.40      0.39      7719
weighted avg       0.40      0.41      0.40      7719

모델 저장 중...
완료!


In [84]:
# 레이블 인코더 생성 및 적용
label_encoder = LabelEncoder()
encoded_emotions = label_encoder.fit_transform(data['Emotion'])

# 레이블 인코더 저장
joblib.dump(label_encoder, 'label_encoder.joblib')

['label_encoder.joblib']

In [1]:
import joblib
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# BERT 모델 로드
class BertFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('skt/kobert-base-v1')
        self.bert.eval()

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state.mean(dim=1)

# 디바이스 설정
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = BertFeatureExtractor().to(device)

# 특징 추출 함수
def get_embeddings(texts, model, device):
    model.eval()
    tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=32)
    
    all_embeddings = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="특징 추출 중", unit="batch"):
            input_ids, attention_mask = [t.to(device) for t in batch]
            embeddings = model(input_ids, attention_mask)
            all_embeddings.append(embeddings.cpu())
    
    return torch.cat(all_embeddings, dim=0).numpy()

# 저장된 파이프라인 모델 불러오기
pipeline = joblib.load('./naive_bayes_model/naive_bayes_pipeline.joblib')

# 레이블 인코더 불러오기 (이전에 저장했다고 가정)
label_encoder = joblib.load('./naive_bayes_model/label_encoder.joblib')

# 테스트 함수
def predict_emotion(text, pipeline, model, device):
    # 텍스트를 임베딩으로 변환
    embedding = get_embeddings([text], model, device)
    # 감정 예측
    prediction = pipeline.predict(embedding)
    # 예측된 레이블을 원래 감정으로 변환
    predicted_emotion = label_encoder.inverse_transform(prediction)[0]
    return predicted_emotion



/Users/leehwan/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 테스트용 문장들
test_sentences = [
    "오늘 날씨가 정말 좋아서 기분이 너무 좋아요!",
    "시험에 떨어져서 너무 실망했어요.",
    "친구와 오랜만에 만나서 즐거운 시간을 보냈습니다.",
    "갑자기 비가 와서 우산을 안 가져온 게 후회돼요.",
    "새로운 프로젝트를 시작하게 되어 기대되고 설레요."
]

print("\n테스트 시작:")
for sentence in test_sentences:
    emotion = predict_emotion(sentence, pipeline, model, device)
    print(f"문장: {sentence}")
    print(f"예측된 감정: {emotion}\n")


테스트 시작:


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
특징 추출 중: 100%|██████████| 1/1 [00:00<00:00, 11.51batch/s]


문장: 오늘 날씨가 정말 좋아서 기분이 너무 좋아요!
예측된 감정: 행복



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
특징 추출 중: 100%|██████████| 1/1 [00:00<00:00, 56.37batch/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


문장: 시험에 떨어져서 너무 실망했어요.
예측된 감정: 슬픔



...

문장: 친구와 오랜만에 만나서 즐거운 시간을 보냈습니다.
예측된 감정: 행복



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
특징 추출 중: 100%|██████████| 1/1 [00:00<00:00, 16.74batch/s]


문장: 갑자기 비가 와서 우산을 안 가져온 게 후회돼요.
예측된 감정: 슬픔



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
특징 추출 중: 100%|██████████| 1/1 [00:00<00:00, 37.08batch/s]

문장: 새로운 프로젝트를 시작하게 되어 기대되고 설레요.
예측된 감정: 행복

